# 2章 パフォーマンスの分析

## 2.2 実行時間の計測

In [ ]:
def slow_way_to_calculate_mode(list_of_numbers):  # 最頻値を計算する遅い方法
    result_dict = {}
    for i in list_of_numbers:
        if i not in result_dict:
            result_dict[i] = 1
        else:
            result_dict[i] += 1

    mode_vals = []
    max_frequency = max(result_dict.values())
    for key, value in result_dict.items():
        if value == max_frequency:
            mode_vals.append(key)

    return mode_vals

In [ ]:
slow_way_to_calculate_mode([4, 5, 5, 6])

In [ ]:
#  「pip install numpy」が必要
import numpy as np

random_integers = np.random.randint(1, 1_000_000, 1_000_000)

In [ ]:
import time

start = time.time()
slow_way_to_calculate_mode(random_integers)
end = time.time()

print(end - start)

In [ ]:
%%timeit
slow_way_to_calculate_mode(random_integers)

In [ ]:
# time_mode.py
import numpy as np
import timeit

random_integers = np.random.randint(1, 100_000, 1000)


def slow_way_to_calculate_mode(list_of_numbers):
    result_dict = {}
    for i in list_of_numbers:
        if i not in result_dict:
            result_dict[i] = 1
        else:
            result_dict[i] += 1
    mode = max(result_dict, key=result_dict.get)
    if result_dict[mode] == 1:
        return min(list_of_numbers)
    else:
        return mode


mode_timer = timeit.Timer(
    stmt="slow_way_to_calculate_mode(random_integers)",
    setup="from __main__ import slow_way_to_calculate_mode, random_integers",
)

time_taken = mode_timer.timeit(number=10)

print(f"Execution time: {time_taken} seconds")

#### ●最頻値を計算する別の方法

In [ ]:
from collections import Counter

def mode_using_counter(list_of_numbers):
    c = Counter(list_of_numbers)
    return c.most_common(1)[0][0]

In [ ]:
%%timeit
mode_using_counter(random_integers)

## 2.3　プロファイリング

### 2.3.1 `cProfile`

In [ ]:
from collections import Counter
import numpy as np

In [ ]:
def mode_using_counter(n_integers):
    random_integers = np.random.randint(1, 100_000, n_integers)
    c = Counter(random_integers)
    return c.most_common(1)[0][0]

In [ ]:
mode_using_counter(10_000_000)

In [ ]:
%%prun
mode_using_counter(10_000_000)

##### メモ： 「`pip install snakeviz`」を実行するとsnakevizをインストールできます

In [ ]:
%load_ext snakeviz

In [ ]:
%%snakeviz
mode_using_counter(10000)

### 2.3.2 `line_profiler`

##### メモ： 「`pip install line_profiler`」でインストール

In [ ]:
%load_ext line_profiler

In [ ]:
%lprun -f mode_using_counter mode_using_counter(10_000_000)

### 2.3.3 Memrayによるメモリプロファイリング

##### メモ： 「`pip install memray`」でインストール

In [ ]:
# mode_using_counter.py
import numpy as np
from collections import Counter

def mode_using_counter(n_integers):
    random_integers = np.random.randint(1, 100_000, n_integers)
    c = Counter(random_integers)
    return c.most_common(1)[0][0]

if __name__ == '__main__':
    print(mode_using_counter(10_000_000))

In [ ]:
!memray run mode_using_counter.py

In [ ]:
!memray flamegraph memray-mode_using_counter.py.8331.bin

メモ： 「8331」の部分は変えて実行してください

In [ ]:
!open memray-flamegraph-mode_using_counter.py.8331.html

In [ ]:
%load_ext memray

In [ ]:
%%memray_flamegraph --trace-python-allocators --leaks
def a():
    return "a" * 10_000

def bc():
    return "bc" * 10_000

x = a() + bc()

## 2.4 時間計算量

In [ ]:
def weighted_mean(list_of_numbers, weights):
    running_total = 0
    for i in range(len(list_of_numbers)):
        running_total += (list_of_numbers[i] * weights[i])
    return (running_total/sum(weights))

In [ ]:
def covariance_fast(X, Y):
    avg_X = sum(X) / len(X)
    avg_Y = sum(Y) / len(Y)

    result = 0
    for i in range(len(X)):
        result += (X[i] - avg_X) * (Y[i] - avg_Y)

    return result / len(X)

In [ ]:
X = np.random.randint(1, 1000, 1000)
Y = np.random.randint(1, 1000, 1000)

In [ ]:
%%timeit
covariance_fast(X, Y)

In [ ]:
def covariance(X, Y):
    cov_sum = 0
    for i in range(len(X)):
        for j in range(len(Y)):
            cov_sum += 0.5 * (X[i] - X[j]) * (Y[i] - Y[j])
    return cov_sum / (len(X) ** 2)

In [ ]:
%%timeit
covariance(X, Y)

## 図2.3（英語版）を生成するコード

##### メモ： 「`pip install matplotlib`」でインストール

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
n = np.linspace(1, 10, 1000)
line_names = [
    "Constant",
    "Linear",
    "Quadratic",
    "Exponential",
    "Logarithmic",
    "N log N",
]

colors = ["black", "orange", "green", "blue", "red", [0.5, 0.5, 0.5]]
linestyles = ["solid", "solid", (0, (5, 5)), "dotted", "dashdot", (0, (5, 1))]
big_o = [np.ones(n.shape), n, n**2, 2**n, np.log(n), n * (np.log(n))]

fig, ax = plt.subplots()
fig.set_facecolor("white")

ax.set_ylim(0, 50)
ax.set_xlim(1, 10)
for i in range(len(big_o)):
    ax.plot(n, big_o[i], label=line_names[i], color=colors[i], linestyle=linestyles[i])
ax.set_ylabel("Relative Runtime")
ax.set_xlabel("Input Size")
ax.legend()
fig.savefig("seds_0202_v3.png", bbox_inches="tight")